# PROJETO 2

## <font color='red'>Carlos Ludman, Felipe Viacava e Yago Ribeiro Labate</font>

### Este trabalho consiste em comparar o tempo médio que o usuário leva para encontrar determinado projeto entre duas interfaces diferentes, sendo elas A e B.
### As interfaces comparadas foram retiradas de uma página de projetos de eletrônica, contida no portfolio realizado pelos alunos Carlos Ludman, Bruno Porto, Henrique Tortorella e Jõao Rodrigues.
### As interfaces comparadas são basicamente iguais, porém com uma pequena diferença. Os projetos de eletrônica da interface A estão localizados ao lado esquerdo da página. Já na interface B, os projetos estão localizados ao lado direito da página, o que leva ao objetivo desse projeto. 
### O intuito desse projeto é determinar quanto tempo o usuário leva, em média, para encontrar algo localizado ao lado esquerdo da interfrace, e comparar com o tempo que ele leva para encontrar esse mesmo projeto, localizado ao lado direito da interface. Assim, será possível concluir se existe, ou não, um lado no qual o usuário demora menos tempo para encontrar o que ele deseja.




In [2]:
import pandas as pd
import json
import itertools
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

### Leitura do arquivo

In [3]:
arquivo = open("teste-ab-c4409-export.json")
linhas = arquivo.readlines()
texto = "".join(linhas)
dados_fire = json.loads(texto)

### Ánalise do atributo logs

In [4]:
for k,v in dados_fire["logs"].items():
    # print(k,v) # habilite para imprimir as chaves de mais alto nível
    for chave, valor in v.items():
        # print(chave, valor) # habilite para imprimir as chaves inferiores
        pass # desabilite quando reabilitar a linha de cima


### Convertendo para um dataframe

In [5]:
lista_dicionarios = []

for k,v in dados_fire["logs"].items():
    lista_dicionarios.append(v)

In [6]:
dataframe = pd.DataFrame.from_records(lista_dicionarios)

In [7]:
dataframe

,tempo,tipo,user_id,versao
0,1478545061371,inicio,3810329911,B
1,1478548168964,inicio,3282163375,B
2,1478545475312,inicio,2480978140,A
3,1478556180853,inicio,4250897646,A
4,1478611317394,inicio,721600819,A
5,1478595097681,inicio,3299522247,A
6,1478565709149,final,1953900235,A
7,1478542505838,final,1468448914,B
8,1478547120700,inicio,1469601225,A
9,1478559093798,final,1451785770,B


### Convertendo para um DataFrame de delta_t

In [8]:
agrupado = dataframe.groupby(by=["user_id", "versao"])

In [9]:
users = set(dataframe.user_id) # We convert it to a set so that each user appear only once

In [10]:
users = list(users) 

In [11]:
resultados = []

for u in users:
    for versao in ["A", "B"]:
        if (u, versao) in agrupado.groups.keys(): # Check if a specific pair user/type of test happened
            grupo1 =  agrupado.get_group((u, versao))
            grupo1 = grupo1.sort_values(by="tempo") # Para evitar que inícios sem fim gerem tempos negativos
            
            if len(grupo1[grupo1.tipo=="final"])>= 1 and len(grupo1[grupo1.tipo=="inicio"]) >= 1:
                delta_t = grupo1[grupo1.tipo=="final"].iloc[-1].tempo - grupo1[grupo1.tipo=="inicio"].iloc[0].tempo
                resultados.append({"user_id":u, "delta_t":delta_t, "versao":versao})

In [12]:
grupo1.sort_values(by="tempo")

,tempo,tipo,user_id,versao
59,1478563961023,inicio,959534589,B
121,1478563999751,final,959534589,B


In [13]:
dataframe_resumo = pd.DataFrame.from_records(resultados)

In [14]:
dataframe_resumo.delta_t/=1000 # Convertendo milissegundos em segundos

In [15]:
dataframe_resumo

,delta_t,user_id,versao
0,3905.814,542355478,B
1,8.389,2830537766,A
2,20.220,583742503,B
3,36.527,1451785770,B
4,157.134,1931315283,A
5,52.763,2484197987,B
6,32.759,1770719342,B
7,17.166,212358778,A
8,21.564,3146881664,B
9,5.582,1561760900,B


In [16]:
a = dataframe_resumo[dataframe_resumo.versao == "A"]
b = dataframe_resumo[dataframe_resumo.versao == "B"]

### Removendo outliers 

In [20]:
tempo_a = a.delta_t.drop(a.index[[4,5]])
tempo_b = b.delta_t.drop(b.index[[0,8]])

### Após o processo de extração dos dados do firebase, foram obtidos os seguintes resultados: 

** Número de termos da amostra A (Interface A): ** 21

** Número de termos da amostra B (Interface B): ** 27

** Média da amostra A (Interface A): ** 51.07680952380953

** Média da amostra B (Interface B): ** 56.605555555555554

** Desvio Padrão da amostra A (Interface A): ** 57.034020692582295

** Desvio Padrão da amostra B (Interface B): ** 74.35498220042584

# Comparação de variâncias populacionais das duas interfaces, sendo as popupalações, os tempos levados para encontrar o projeto "Estação meteorológica". ##

In [21]:
nA = tempo_a.count() # Número de termos da população A
nB = tempo_b.count() # Número de termos da população B
xA_barra = tempo_a.mean() # Média de tempo da população A
xB_barra = tempo_b.mean() # Média de tempo da população B
S_A = tempo_a.std() # Desvio padrão do tempo da população A
S_B = tempo_b.std() # Desvio padrão do tempo da população B
alpha = 0.1 # Nível de significância determinado

### Técnica inferencial: 

Teste de hipóteses, cuja a hipótese nula consistirá no desvio padrão da população A ser igual ao da população B

### Hipóteses: 

**Ho:** σA = σB

**Ha:** σA ≠ σB


### Região não crítica: (F1<Região não crítica<F2)

In [22]:
grau_de_liberdadeA = nA -1
grau_de_liberdadeB = nB -1

print("Para a distribuição F(",grau_de_liberdadeA,",",grau_de_liberdadeB,") ao nível de significância 5%, a região não crítica se encontra, segundo a tabela da distribuição F, no intervalo ]0.484,1.990[")

Para a distribuição F( 20 , 26 ) ao nível de significância 5%, a região não crítica se encontra, segundo a tabela da distribuição F, no intervalo ]0.484,1.990[


### Determinando o F observado:


In [23]:
F_observado = (S_B**2)/(S_A**2)
print('O F observado é',F_observado)

O F observado é 1.6996213201923402


**Como F1<F<F2, ou seja, o F observado encontra-se na região de aceitação, não há evidências para rejeitar a hipótese nula
**

# Comparação das médias populacionais das duas interfaces, sendo as popupalações, os tempos levados para encontrar o projeto "Estação meteorológica". ##

### Hipóteses:

**Ho:**  μA =  μB

**Ha:**  μA ≠  μB



 ### Região não crítica: (-tc<Região não crítica<tc)

In [29]:
grau_de_liberdade = nA + nB - 2
t = stats.t.ppf(alpha,grau_de_liberdade)
print("O t(",grau_de_liberdade,') é, segundo a tabela de distribuição t,',t)
print("Portanto, a região crítica se encontra entre ]",t,-1*t,"[")

O t( 46 ) é, segundo a tabela de distribuição t, -1.30022804738
Portanto, a região crítica se encontra entre ] -1.30022804738 1.30022804738 [


### Determinando o T observado:

In [26]:
S_quadrado = (((nA - 1)*S_A**2) + ((nB - 1)*S_B**2))/(nA + nB - 2)

T = (xA_barra - xB_barra)/((S_quadrado/nA)+(S_quadrado/nB))**(1/2)

print('O T é',T)

O T é -0.282038417037


**Como -tc<T<tc, ou seja, o T observado não encontra-se na região de crítica, não há evidências para rejeitar a hipótese nula.
**

## Conclusão:

** Visto que a hipótese nula não pode ser rejeitada, ou seja, que não há evidências para rejeitar a hipótese de que as médias populacionais são iguais, conclui-se que o tempo no qual um usuário leva para encontrar algo localizado ao lado direito da interface, é o mesmo levado para encontrá-lo caso estivesse localizado ao lado esquerdo da interface.  ** 